In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

#../om0.1000_w-1.0000/10.csv.shiftz_nbar1e-3_output_numNNB200_beta3.0000/om0.3071_w-1.0000.BSKinfo.length.numbers
#../om0.1000_w-1.0000/10.csv.shiftz_nbar1e-3_output_numNNB200_beta3.0000/om0.3071_w-1.0000.BSKinfo.mu.numbers

zfile2 = "/home/xiaodongli/data/BigMDPL/snaps/nbar1e-3/Redshifts.dat"
zdata = np.loadtxt(zfile2)
snapz = {}
snaps = np.array([10, 12, 15, 20, 26, 33, 41, 48, 58, 75])
zs = np.zeros(len(snaps))
for X in zdata:
    for row in range(len(snaps)):
        if abs(X[0] - snaps[row]) < 0.01:
            #z = X[3]; z = max(z,0.001) # avoid redshift 0...
            zs[row] = X[3]
            snapz[snaps[row]] = zs[row]
print('Relation between the snapid and redshift: \n\t',snapz)

def edgefile(om, w, snpid, shiftz=True,quantity='length',beta=3):
    omwstr = 'om%.4f'%om+'_w%.4f'%w
    if shiftz:
        return "../"+omwstr+"/"+str(snpid)+".csv.shiftz_nbar1e-3_output_numNNB200_beta"+str(beta)+".0000/om0.3071_w-1.0000.BSKinfo."+quantity+".edges"
    else:
        return "../"+omwstr+"/"+str(snpid)+".csv.nbar1e-3_output_numNNB200_beta"+str(beta)+".0000/om0.3071_w-1.0000.BSKinfo."+quantity+".edges"

def numberfile(om, w, snpid, shiftz=True,quantity='length',beta=3):
    omwstr = 'om%.4f'%om+'_w%.4f'%w
    if shiftz:
        return "../"+omwstr+"/"+str(snpid)+".csv.shiftz_nbar1e-3_output_numNNB200_beta"+str(beta)+".0000/om0.3071_w-1.0000.BSKinfo."+quantity+".numbers"
    else:
        return "../"+omwstr+"/"+str(snpid)+".csv.nbar1e-3_output_numNNB200_beta"+str(beta)+".0000/om0.3071_w-1.0000.BSKinfo."+quantity+".numbers"

def histfiles(om, w, snpid,shiftz=True,quantity='length',beta=3):
    return edgefile(om,w,snpid,shiftz,quantity,beta), numberfile(om,w,snpid,shiftz,quantity,beta)
def Xmid(x):
    return (x[:len(x)-1]+x[1:]) / 2.
def fitpeak(x,y,n=5, deg=10, return_xy=False):
        #print('(fitpeak) len(x), len(y) =  ', len(x),len(y))
        ymax = max(y)
        iymax = np.where(y == ymax)[0][0]
        x2, y2 = x[iymax-n:iymax+n], y[iymax-n:iymax+n]
        #print('(fitpeak) len(x2), len(y2) =  ', len(x2),len(y2))

        if True:
            poly = np.polyfit(x2,y2,deg)
            xs = np.linspace(x2[0], x2[len(x2)-1], 10000)
            ys = np.polyval(poly,xs)
            ymax = max(ys)
            iymax = np.where(ys == ymax)[0][0]
            xpeak = xs[iymax]
        if not return_xy:
            return xpeak
        else:
            return xpeak, xs, ys

def groupX(A,n=10):
    maxrow = int(len(A) / n) * n 
    rlt = np.zeros(int(maxrow/n))
    for row in range(n):
        rlt = rlt + A[row:maxrow:n]
    rlt = rlt / n
    return rlt

def chisq_dispersion(X):
    xbar = np.mean(X)
    return sum( (X - xbar)**2.0 )



In [ ]:
deg = 8; n = 5;
#om_ws = [ [0.6,-1] , [0.3071, -0.5], [0.3071,-1.5], [0.1, -1], [0.3071,-1]  ]
om_ws = [ [0.1, -1], [0.3071,-1]  ]
for beta in [3]:
  for omw in om_ws:
    fig, axs = plt.subplots(1,2,figsize=(16,8))
    om, w = omw
    colors = ['k', 'r', 'b', 'g', 'c', 'y', 'm', 'gray', 'orange']
    maxy = 0
    xpeaks = [[],[]]
    for isnp, snpid in enumerate([75, 58, 48, 41, 33, 26]): #enumerate(snaps[[5,]]):#1,]]):
#snaps = np.array([10, 12, 15, 20, 26, 33, 41, 48, 58, 75])

        redshift = snapz[snpid]; c = colors[isnp]
        for iquantity, quantity in enumerate(['length', 'mu']):
            xfile, yfile = histfiles(om,w,snpid,True,quantity,beta)
            print('Load in: xfile, yfile = ', xfile, yfile)
            x, y = np.loadtxt(xfile), np.loadtxt(yfile); x = abs(x); x = Xmid(x)
            print('length of x, y = ',len(x), len(y))
            if iquantity == 1:
                x = groupX(x,30); y = groupX(y,30)
            else:
                x = groupX(x,10); y = groupX(y,10)

            ax = axs[iquantity]
            for degL, nL, degmu, nmu in [[4,10,3,8]]:#, [8,5], [8,8]]:
                if iquantity == 0:
                    deg, n = degL, nL
                elif iquantity == 1:
                    deg, n = degmu, nmu
                y = y/max(y); 
                ax.plot(x, y, marker = 'o', c=c, markersize=2, lw=0.3)
                if iquantity == 0:
                    xpeak, xs, ys = fitpeak(x,y, n=n, deg=deg, return_xy = True); #ys = ys/max(ys)
                    xpeaks[iquantity].append(xpeak)
                    ax.plot(xs,ys, label='z = %.3f'%redshift+', xpeak = %.3f'%xpeak,c=c,lw=2)
                    ax.plot([xpeak, xpeak,], [0, max(ys)*1.1], lw=2, ls='--', c=c)
                    maxy = max([maxy, max(ys)])
            if iquantity == 0:
                ax.set_xlim(2,3); ax.set_ylim(0, maxy*1.2); ax.set_xlabel('Length', fontsize=14)
            else:
                ax.set_xlim(0,1); ax.set_ylim(0.4, 1.2); ax.set_xlabel('$\\mu$', fontsize=14)
    chisqs = map(chisq_dispersion, [np.array(xpeaks[0]), np.array(xpeaks[1])])
    for iax, ax in enumerate(axs):
            ax.legend(frameon=False,fontsize=14)
            if iax == 0:
                    deg, n = degL, nL
            elif iax == 1:
                    deg, n = degmu, nmu
            ax.set_title(' deg, n = '+str([deg,n])+'\n 1000*chisq = %.3f'%(1000*chisqs[iax]),fontsize=14)

    fig.suptitle('om = %.2f'%om+' w = %.2f'%w+'      $\\beta='+str(beta)+'$', fontsize=16)
    fig.savefig('om%.2f'%om+'_w%.2f'%w+'_beta'+str(beta)+'.png', format='png')
    plt.show()